<a href="https://colab.research.google.com/github/William-Metz/NXTPlaySampleNotebooks/blob/main/FormationDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this notebook import the data set with player coorindates and formations, uses keras to try to predict the formation based on the coordinates

# Installation and Setup


In [ ]:
!pip install keras-tuner
!pip install tensorflow-gpu
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.regularizers import l1
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras_tuner.tuners import RandomSearch
!pip install keras-tuner
!pip install tensorflow-gpu

import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Data
In order to make this notebook work, make sure to have the dataset with player coordinates


In [ ]:
df = pd.read_csv('/content/drive/My Drive/MidtermProject/simplifiedDataWithCoordinates.csv')

columns_to_keep = [f'offense_player{i}_x' for i in range(1, 12)] + \
                  [f'offense_player{i}_y' for i in range(1, 12)] +\
                  [f'defense_player{i}_x' for i in range(1, 12)] + \
                  [f'defense_player{i}_y' for i in range(1, 12)] + \
                   ['off_formation']




df = df[columns_to_keep]

# Replace 0.0 with numpy.nan
df = df.replace(0.0, np.nan)

# Drop rows with NaN in specific columns
#df = df.dropna(subset=['offense_player1_x', 'defense_player2_x', 'defense_player2_x', 'defense_player1_x'])
#Drops empty players
df = df.dropna(subset=['offense_player7_x', 'defense_player7_x'])
# Replace NaN back with 0.0 if necessary
df = df.fillna(0.0)


This was done unifomize the number of each formation in the dataset.

In [ ]:
dic = {}
for index, row in df.iterrows():
    # access data using row['column_name']
    dic[row['off_formation']] = dic.get(row['off_formation'], 0) + 1
print(df.shape)
print(dic)
for key, value in dic.items():
    if value < 20:
        # Drop rows where 'off_formation' equals key
        df.drop(df[df['off_formation'] == key].index, inplace=True)
    else:
        # Get indices where 'off_formation' equals key
        indices = df[df['off_formation'] == key].index
        print(len(indices))
        # Randomly select 19 or fewer indices from this list
        indices_to_drop = np.random.choice(indices, size= len(indices) -20 , replace=False)

        # Drop the selected rows
        df.drop(indices_to_drop, inplace=True)
print(len(dic))

(372, 45)
{'B FLEX': 21, 'LIGHTNING': 25, 'CHIEF': 6, 'KINGS SPLIT RIGHT ': 73, 'KINGS SPLIT': 11, 'KINGS TIGHT': 3, 'SPACE': 2, 'SPLIT': 13, 'SPLIT FLEX': 2, 'LIGHTNING SQUEEZE': 3, 'PLANE FLEX': 9, 'BENGALS': 6, 'STRONG SPLIT': 1, 'STRONG FLEX': 1, 'CHARGER': 2, 'CINCINNATI': 1, 'KINGS SPLIT LEFT': 60, 'LIGHTNING STACK': 2, 'EMPTY': 14, 'LIGHTNING TITE LEFT': 31, 'UNBALANCED ': 1, 'TWINS SPLIT LEFT': 6, 'ACES LEFT ': 4, 'LIGHTNING TITE RIGHT': 35, 'TWINS SPLIT RIGHT': 7, 'QUEENS LEFT': 9, 'QUEENS RIGHT': 15, 'ACES Right': 4, 'KINGS RIGHT': 5}
21
25
73
60
31
35
29


Split into testing and training

In [ ]:

df = pd.get_dummies(df, columns=['off_formation'])
X = df.drop([col for col in df.columns if col.startswith('off_formation')], axis=1)
y = df[[col for col in df.columns if col.startswith('off_formation')]]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Training and testing different models

In [ ]:

def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('input_units', min_value=32, max_value=256, step=32),
                    activation='relu', input_shape=(X_train_scaled.shape[1],)))

    for i in range(hp.Int('n_layers', 1, 4)):
        model.add(Dense(hp.Int(f'dense_{i}_units', min_value=32, max_value=256, step=32),
                        activation='relu', kernel_regularizer=l1(hp.Choice('l1_reg', values=[0.01, 0.001, 0.0001]))))
        model.add(Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(y_train.shape[1], activation='softmax'))

    model.compile(optimizer=Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Number of trials to run
    executions_per_trial=1,  # Number of models to train per trial
    directory='my_dir',
    project_name='hparam_tuning'
)
with tf.device('/GPU:0'):
  tuner.search(X_train_scaled, y_train, epochs=100, validation_split=0.2)
  best_model = tuner.get_best_models(num_models=1)[0]
  loss, accuracy = best_model.evaluate(X_test_scaled, y_test)
  print(f"Test Accuracy: {accuracy}")


Trial 10 Complete [00h 00m 06s]
val_accuracy: 0.25

Best val_accuracy So Far: 0.3499999940395355
Total elapsed time: 00h 01m 03s
1/1 [==============================] - 0s 292ms/step - loss: 2.6165 - accuracy: 0.2083
Test Accuracy: 0.2083333283662796


In [ ]:

for i in X_train_scaled:

  # Reshape the data for the model, if necessary
  predictions = best_model.predict(i.reshape(1, -1))
  predicted_class_index = np.argmax(predictions, axis=1)
  text = y.columns[predicted_class_index][0]  # G
  print(text)


1/1 [==============================] - 0s 22ms/step
off_formation_KINGS SPLIT LEFT
1/1 [==============================] - 0s 21ms/step
off_formation_EMPTY
1/1 [==============================] - 0s 21ms/step
off_formation_LIGHTNING TITE LEFT
1/1 [==============================] - 0s 22ms/step
off_formation_KINGS SPLIT RIGHT 
1/1 [==============================] - 0s 23ms/step
off_formation_LIGHTNING TITE RIGHT
1/1 [==============================] - 0s 21ms/step
off_formation_LIGHTNING
1/1 [==============================] - 0s 22ms/step
off_formation_KINGS SPLIT RIGHT 
1/1 [==============================] - 0s 20ms/step
off_formation_KINGS SPLIT LEFT
1/1 [==============================] - 0s 24ms/step
off_formation_KINGS SPLIT LEFT
1/1 [==============================] - 0s 21ms/step
off_formation_LIGHTNING
1/1 [==============================] - 0s 21ms/step
off_formation_LIGHTNING TITE LEFT
1/1 [==============================] - 0s 22ms/step
off_formation_LIGHTNING TITE RIGHT
1/1 [======

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

# Assuming X_train_scaled is already defined and is a 2D NumPy array
n_features = X_train_scaled.shape[1]  # Number of features in your dataset
n_players = 22  # Number of players
n_coordinates = 2  # XY coordinates
n_channels = 1  # Single channel
# Reshape the data for Conv1D layer
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], n_players, n_coordinates))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], n_players, n_coordinates))
print(X_train_reshaped.shape)
print(X_test_reshaped.shape)



model = Sequential([
    # First Conv1D layer
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_players, n_coordinates)),
    MaxPooling1D(pool_size=2),

    # Additional Conv1D layers (you can add more or modify as needed)
    Conv1D(filters=128, kernel_size=2, activation='relu'),
    MaxPooling1D(pool_size=2),

    # Flatten the output before feeding it into dense layers
    Flatten(),

    # Dense layers for classification
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y_train.shape[1], activation='softmax')  # Update with your actual number of classes
])

learning_rate = 0.001  # Experiment with different rates
optimizer = SGD(learning_rate=learning_rate, momentum= .8)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Train the model
history = model.fit(X_train_reshaped, y_train, validation_split=0.2, epochs=400, batch_size=30, callbacks=[early_stopping])
test_loss, test_accuracy = model.evaluate(X_test_reshaped, y_test)
print(f"Test Accuracy: {test_accuracy}")

(96, 22, 2)
(24, 22, 2)
Epoch 1/400
3/3 [==============================] - 1s 122ms/step - loss: 1.8098 - accuracy: 0.1711 - val_loss: 1.7459 - val_accuracy: 0.3500
Epoch 2/400
3/3 [==============================] - 0s 19ms/step - loss: 1.8475 - accuracy: 0.1316 - val_loss: 1.7452 - val_accuracy: 0.3500
Epoch 3/400
3/3 [==============================] - 0s 18ms/step - loss: 1.8081 - accuracy: 0.1711 - val_loss: 1.7449 - val_accuracy: 0.3500
Epoch 4/400
3/3 [==============================] - 0s 19ms/step - loss: 1.8182 - accuracy: 0.1842 - val_loss: 1.7438 - val_accuracy: 0.3500
Epoch 5/400
3/3 [==============================] - 0s 19ms/step - loss: 1.7861 - accuracy: 0.1842 - val_loss: 1.7429 - val_accuracy: 0.3000
Epoch 6/400
3/3 [==============================] - 0s 19ms/step - loss: 1.8067 - accuracy: 0.1711 - val_loss: 1.7424 - val_accuracy: 0.2500
Epoch 7/400
3/3 [==============================] - 0s 18ms/step - loss: 1.8056 - accuracy: 0.1447 - val_loss: 1.7425 - val_accuracy: 0.

In [ ]:
for i in X_train_scaled:
  new_input_scaled = scaler.transform(i.reshape(1, -1))

  # Reshape the data for the model, if necessary
  input_data_reshaped = new_input_scaled.reshape((1, 11, 2))
  predictions = model.predict(input_data_reshaped)
  predicted_class_index = np.argmax(predictions, axis=1)
  text = y.columns[predicted_class_index][0]  # G
  print(text)

## Saving and exporting the model

In [ ]:
import joblib

# Save the scaler
joblib.dump(scaler, 'my_scaler.pkl')
model.save('my_model.h5')  # Save as HDF5
# or
model.save('my_saved_model')  # Save as SavedModel

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
